1) Master File Splitting

In [20]:
import pandas as pd
import os
from datetime import datetime

# File path
file_path = r'D:\Yash\Project\Raw File\credit_card_transactions Master Dataset.csv'

# Output folder
output_folder = r'D:\Yash\Project\Outputs'
os.makedirs(output_folder, exist_ok=True)

# Chunk size for reading
chunksize = 100000

In [21]:
category_cols = ['category']
categories = pd.DataFrame()

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunk.columns = chunk.columns.str.strip().str.lower().str.replace(" ", "_")
    chunk_cat = chunk[['category']].drop_duplicates()
    categories = pd.concat([categories, chunk_cat], ignore_index=True)

categories.drop_duplicates(subset=['category'], inplace=True)
categories['category_id'] = range(1, len(categories)+1)
categories.rename(columns={'category': 'category_name'}, inplace=True)
categories = categories[['category_id', 'category_name']]

# Save to CSV
categories.to_csv(os.path.join(output_folder, 'categories.csv'), index=False)
print("Categories CSV saved.")


Categories CSV saved.


In [22]:
merchant_cols = ['merchant','category','merch_lat','merch_long','merch_zipcode']
merchants = pd.DataFrame()

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunk.columns = chunk.columns.str.strip().str.lower().str.replace(" ", "_")
    chunk_merchants = chunk[merchant_cols].drop_duplicates(subset=['merchant'])
    merchants = pd.concat([merchants, chunk_merchants], ignore_index=True)

merchants.drop_duplicates(subset=['merchant'], inplace=True)

# Map category_id from categories table
merchants = merchants.merge(categories, left_on='category', right_on='category_name', how='left')

# Assign merchant_id
merchants['merchant_id'] = range(1, len(merchants)+1)
merchants.rename(columns={
    'merchant': 'merchant_name',
    'merch_lat': 'merchant_lat',
    'merch_long': 'merchant_long',
    'merch_zipcode': 'merchant_zipcode'
}, inplace=True)

merchants = merchants[['merchant_id','merchant_name','category_id','merchant_lat','merchant_long','merchant_zipcode']]

# Save to CSV
merchants.to_csv(os.path.join(output_folder, 'merchants.csv'), index=False)
print("Merchants CSV saved.")


Merchants CSV saved.


In [23]:
date_table = pd.DataFrame()

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunk.columns = chunk.columns.str.strip().str.lower().str.replace(" ", "_")
    chunk['trans_date'] = pd.to_datetime(chunk['trans_date_trans_time'].str.split(' ').str[0], errors='coerce')
    date_table = pd.concat([date_table, chunk[['trans_date']]], ignore_index=True)

date_table.drop_duplicates(subset=['trans_date'], inplace=True)
date_table['date_id'] = date_table['trans_date']
date_table['year'] = date_table['trans_date'].dt.year
date_table['quarter'] = 'Q' + date_table['trans_date'].dt.quarter.astype(str)
date_table['month_number'] = date_table['trans_date'].dt.month
date_table['month_name'] = date_table['trans_date'].dt.month_name()
date_table['week_number'] = date_table['trans_date'].dt.isocalendar().week
date_table['day_of_week'] = date_table['trans_date'].dt.day_name()
date_table['is_weekend'] = date_table['day_of_week'].isin(['Saturday','Sunday'])

date_table = date_table[['date_id','year','quarter','month_number','month_name','week_number','day_of_week','is_weekend']]

# Save to CSV
date_table.to_csv(os.path.join(output_folder, 'date_table.csv'), index=False)
print("Date table CSV saved.")


Date table CSV saved.


In [24]:
import pandas as pd
import os

# File path and output folder
file_path = r'D:\Yash\Project\Raw File\credit_card_transactions Master Dataset.csv'
output_folder = r'D:\Yash\Project\Outputs'
os.makedirs(output_folder, exist_ok=True)

chunksize = 100000

def normalize_columns(df):
    return df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"))

# Columns to extract for customers
customer_cols = ['cc_num','first','last','gender','street','city','state','zip','lat','long','city_pop','job','dob']
unique_customers = pd.DataFrame(columns=customer_cols)

# Read in chunks and accumulate unique customers
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunk = normalize_columns(chunk)
    chunk_customers = chunk[customer_cols].drop_duplicates(subset=['cc_num'])
    unique_customers = pd.concat([unique_customers, chunk_customers], ignore_index=True)

# Deduplicate across all chunks
unique_customers.drop_duplicates(subset=['cc_num'], inplace=True)

# Assign numeric customer_id
unique_customers['customer_id'] = range(1, len(unique_customers)+1)

# Save helper mapping table (customer_id ↔ cc_num)
customer_map = unique_customers[['customer_id','cc_num']]
customer_map.to_csv(os.path.join(output_folder, 'customer_map.csv'), index=False)
print("Customer helper table saved: customer_map.csv")

# Generate final customers dimension table
unique_customers['full_name'] = unique_customers['first'] + ' ' + unique_customers['last']
unique_customers['dob'] = pd.to_datetime(unique_customers['dob'], errors='coerce')
unique_customers['age'] = (pd.Timestamp('today') - unique_customers['dob']).dt.days // 365

unique_customers.rename(columns={
    'zip': 'zipcode',
    'lat': 'latitude',
    'long': 'longitude',
    'city_pop': 'city_population'
}, inplace=True)

customers = unique_customers[['customer_id','full_name','first','last','gender',
                              'street','city','state','zipcode','latitude',
                              'longitude','city_population','job','dob','age']]
customers.to_csv(os.path.join(output_folder, 'customers.csv'), index=False)
print("Customers dimension table saved: customers.csv")


C:\Users\Freshcore2025\AppData\Local\Temp\ipykernel_18724\3651082439.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unique_customers = pd.concat([unique_customers, chunk_customers], ignore_index=True)


Customer helper table saved: customer_map.csv
Customers dimension table saved: customers.csv


In [25]:
import pandas as pd
import os

# Paths
output_folder = r'D:\Yash\Project\Outputs'
file_path = r'D:\Yash\Project\Raw File\credit_card_transactions Master Dataset.csv'
transactions_output = os.path.join(output_folder, 'transactions.csv')
chunksize = 100000

# Load helper/reference tables
customer_map = pd.read_csv(os.path.join(output_folder, 'customer_map.csv'))  # columns: cc_num, customer_id
merchants = pd.read_csv(os.path.join(output_folder, 'merchants.csv'))      # columns: merchant_id, merchant_name
categories = pd.read_csv(os.path.join(output_folder, 'categories.csv'))    # columns: category_id, category_name

# Normalize reference tables for safe merge
merchants['merchant_name'] = merchants['merchant_name'].astype(str).str.strip().str.lower()
categories['category_name'] = categories['category_name'].astype(str).str.strip().str.lower()

transaction_cols = ['trans_num','trans_date_trans_time','cc_num','merchant','category','amt','is_fraud']

is_first_write = True
transaction_id_counter = 1  # numeric transaction ID

def normalize_columns(df):
    return df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"))

for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize)):
    chunk = normalize_columns(chunk)
    tx = chunk[transaction_cols].copy()

    # Map customer_id using helper table
    tx = tx.merge(customer_map, on='cc_num', how='left')
    tx.drop(columns=['cc_num'], inplace=True)

    # Normalize text columns in master file
    tx['merchant'] = tx['merchant'].astype(str).str.strip().str.lower()
    tx['category'] = tx['category'].astype(str).str.strip().str.lower()

    # Merge merchant_id
    tx = tx.merge(merchants[['merchant_id','merchant_name']], 
                  left_on='merchant', right_on='merchant_name', how='left')
    tx.drop(columns=['merchant','merchant_name'], inplace=True)

    # Merge category_id
    tx = tx.merge(categories[['category_id','category_name']], 
                  left_on='category', right_on='category_name', how='left')
    tx.drop(columns=['category','category_name'], inplace=True)

    # Check for missing mappings
    missing_customer = tx['customer_id'].isnull().sum()
    missing_merchant = tx['merchant_id'].isnull().sum()
    missing_category = tx['category_id'].isnull().sum()
    if missing_customer > 0 or missing_merchant > 0 or missing_category > 0:
        print(f"Chunk {i+1} warnings - missing IDs: "
              f"customer={missing_customer}, merchant={missing_merchant}, category={missing_category}")

    # Split transaction datetime
    tx['trans_date_trans_time'] = pd.to_datetime(tx['trans_date_trans_time'], errors='coerce')
    tx['transaction_date'] = tx['trans_date_trans_time'].dt.date
    tx['transaction_time'] = tx['trans_date_trans_time'].dt.time
    tx.drop(columns=['trans_date_trans_time'], inplace=True)

    # Rename amount
    tx.rename(columns={'amt':'amount'}, inplace=True)

    # Assign numeric unique transaction_id
    tx['transaction_id'] = range(transaction_id_counter, transaction_id_counter + len(tx))
    transaction_id_counter += len(tx)

    # Reorder final columns
    final_cols = ['transaction_id','customer_id','merchant_id','category_id',
                  'transaction_date','transaction_time','amount','is_fraud']
    tx = tx[final_cols]

    # Append to CSV
    tx.to_csv(transactions_output, index=False, mode='w' if is_first_write else 'a', header=is_first_write)
    is_first_write = False
    print(f"Chunk {i+1} processed")

print("Transactions fact table saved successfully with numeric transaction_id.")


Chunk 1 processed
Chunk 2 processed
Chunk 3 processed
Chunk 4 processed
Chunk 5 processed
Chunk 6 processed
Chunk 7 processed
Chunk 8 processed
Chunk 9 processed
Chunk 10 processed
Chunk 11 processed
Chunk 12 processed
Chunk 13 processed
Transactions fact table saved successfully with numeric transaction_id.


In [ ]:
""" import pandas as pd
import os

# File path and output folder
file_path = r'D:\Yash\Project\Raw File\credit_card_transactions Master Dataset.csv'
output_folder = r'D:\Yash\Project\Outputs'
os.makedirs(output_folder, exist_ok=True)

chunksize = 100000

def normalize_columns(df):
    return df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"))

# --------------------
# Step 1: Create customer helper table (cc_num -> customer_id)
# --------------------
customer_cols = ['cc_num','first','last','gender','street','city','state','zip','lat','long','city_pop','job','dob']
unique_customers = pd.DataFrame(columns=customer_cols)

for chunk in pd.read_csv(file_path, chunksize=chunksize):
    chunk = normalize_columns(chunk)
    chunk_customers = chunk[customer_cols].drop_duplicates(subset=['cc_num'])
    unique_customers = pd.concat([unique_customers, chunk_customers], ignore_index=True)

unique_customers.drop_duplicates(subset=['cc_num'], inplace=True)
unique_customers['customer_id'] = range(1, len(unique_customers)+1)

# Save helper mapping table
customer_map = unique_customers[['customer_id','cc_num']]
customer_map.to_csv(os.path.join(output_folder, 'customer_map.csv'), index=False)
print("Customer helper table saved: customer_map.csv")

# --------------------
# Step 2: Create final customers.csv (dimension table)
# --------------------
unique_customers['full_name'] = unique_customers['first'] + ' ' + unique_customers['last']
unique_customers['dob'] = pd.to_datetime(unique_customers['dob'], errors='coerce')
unique_customers['age'] = (pd.Timestamp('today') - unique_customers['dob']).dt.days // 365

unique_customers.rename(columns={
    'zip': 'zipcode',
    'lat': 'latitude',
    'long': 'longitude',
    'city_pop': 'city_population'
}, inplace=True)

customers = unique_customers[['customer_id','full_name','first','last','gender',
                              'street','city','state','zipcode','latitude',
                              'longitude','city_population','job','dob','age']]
customers.to_csv(os.path.join(output_folder, 'customers.csv'), index=False)
print("Customers dimension table saved: customers.csv")

# --------------------
# Step 3: Create transactions.csv (fact table) with numeric transaction_id
# --------------------
transaction_cols = [
    'trans_num','trans_date_trans_time','cc_num','merchant','category',
    'amt','unix_time','merch_lat','merch_long','is_fraud'
]

transactions_output = os.path.join(output_folder, 'transactions_old.csv')
is_first_write = True
transaction_id_counter = 1  # numeric transaction ID starting from 1

for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunksize)):
    chunk = normalize_columns(chunk)
    tx = chunk[transaction_cols].copy()

    # Map customer_id using helper table
    tx = tx.merge(customer_map, on='cc_num', how='left')

    # Drop cc_num for privacy
    tx.drop(columns=['cc_num'], inplace=True)

    # Rename columns
    tx.rename(columns={
        'trans_num': 'old_transaction_id',  # optional reference
        'trans_date_trans_time': 'transaction_date',
        'amt': 'amount'
    }, inplace=True)

    # Assign unique numeric transaction_id
    tx['transaction_id'] = range(transaction_id_counter, transaction_id_counter + len(tx))
    transaction_id_counter += len(tx)

    # Reorder columns
    tx = tx[['transaction_id','customer_id','merchant','category','transaction_date',
             'unix_time','merch_lat','merch_long','amount','is_fraud']]

    # Append to CSV
    tx.to_csv(transactions_output, index=False, mode='w' if is_first_write else 'a', header=is_first_write)
    is_first_write = False

    print(f"Chunk {i+1} processed")

print("Transactions fact table saved: transactions.csv")


2) Connct with Database

In [40]:
import psycopg2

try:
    # Attempt to connect
    conn = psycopg2.connect(
        host="localhost",        # or your server
        dbname="cc_analysis",     # your database name
        user="postgres",        # your username
        password="YashSingh@1205" # your password
    )
    print("Connection successful!")
    
except Exception as e:
    print("Connection failed:", e)

finally:
    # Close connection if it was opened
    if 'conn' in locals() and conn:
        conn.close()


Connection successful!


Importing data into categories table

In [39]:
import psycopg2

# Database connection
conn = psycopg2.connect(
    host="localhost",
    dbname="cc_analysis",
    user="postgres",
    password="YashSingh@1205",
    port=5432
)

categories_file = r"D:\Yash\Project\Outputs\categories.csv"

try:
    with conn:
        with conn.cursor() as cur:
            # Load categories CSV into table
            with open(categories_file, 'r', encoding='utf-8') as f:
                cur.copy_expert("COPY categories FROM STDIN WITH CSV HEADER", f)
            
            cur.execute("SELECT COUNT(*) FROM categories;")
            count = cur.fetchone()[0]
            print(f"Categories uploaded successfully: {count} rows.")

except Exception as e:
    print("Upload failed:", e)
finally:
    conn.close()
    print("Database connection closed.")


Categories uploaded successfully: 14 rows.
Database connection closed.


Importing data into customers table

In [41]:
import psycopg2

# Database connection
conn = psycopg2.connect(
    host="localhost",
    dbname="cc_analysis",
    user="postgres",
    password="YashSingh@1205",
    port=5432
)

customers_file = r"D:\Yash\Project\Outputs\customers.csv"

try:
    with conn:
        with conn.cursor() as cur:
            # Load customers CSV into table
            with open(customers_file, 'r', encoding='utf-8') as f:
                cur.copy_expert("COPY customers FROM STDIN WITH CSV HEADER", f)
            
            cur.execute("SELECT COUNT(*) FROM customers;")
            count = cur.fetchone()[0]
            print(f"Customers uploaded successfully: {count} rows.")

except Exception as e:
    print("Upload failed:", e)
finally:
    conn.close()
    print("Database connection closed.")


Customers uploaded successfully: 983 rows.
Database connection closed.


Importing data into Merchants table

In [42]:
import psycopg2

# Database connection
conn = psycopg2.connect(
    host="localhost",
    dbname="cc_analysis",
    user="postgres",
    password="YashSingh@1205",
    port=5432
)

merchant_csv = r"D:\Yash\Project\Outputs\merchants.csv"

try:
    with conn:
        with conn.cursor() as cur:
            # Optional: Clear table first
            cur.execute("TRUNCATE TABLE merchants;")
            
            # Load CSV into table
            with open(merchant_csv, 'r', encoding='utf-8') as f:
                cur.copy_expert("COPY merchants FROM STDIN WITH CSV HEADER", f)
            
            # Row count check
            cur.execute("SELECT COUNT(*) FROM merchants;")
            print(f"Merchants uploaded successfully, {cur.fetchone()[0]} rows.")

except Exception as e:
    print("Upload failed:", e)
finally:
    conn.close()
    print("Database connection closed.")


Merchants uploaded successfully, 693 rows.
Database connection closed.


Importing data into Date table

In [43]:
import psycopg2

# Database connection
conn = psycopg2.connect(
    host="localhost",
    dbname="cc_analysis",
    user="postgres",
    password="YashSingh@1205",
    port=5432
)

date_csv = r"D:\Yash\Project\Outputs\date_table.csv"

try:
    with conn:
        with conn.cursor() as cur:
            # Optional: clear table first
            cur.execute("TRUNCATE TABLE date_table;")
            
            # Load CSV into table
            with open(date_csv, 'r', encoding='utf-8') as f:
                cur.copy_expert("COPY date_table FROM STDIN WITH CSV HEADER", f)
            
            # Row count check
            cur.execute("SELECT COUNT(*) FROM date_table;")
            print(f"Date table uploaded successfully, {cur.fetchone()[0]} rows.")

except Exception as e:
    print("Upload failed:", e)
finally:
    conn.close()
    print("Database connection closed.")


Date table uploaded successfully, 537 rows.
Database connection closed.


Importing data into transactions table

In [44]:
import psycopg2

# Database connection
conn = psycopg2.connect(
    host="localhost",
    dbname="cc_analysis",
    user="postgres",
    password="YashSingh@1205",
    port=5432
)

transactions_csv = r"D:\Yash\Project\Outputs\transactions.csv"

try:
    with conn:
        with conn.cursor() as cur:
            # Optional: clear table first
            cur.execute("TRUNCATE TABLE transactions;")
            
            # Load CSV into table
            with open(transactions_csv, 'r', encoding='utf-8') as f:
                cur.copy_expert("COPY transactions FROM STDIN WITH CSV HEADER", f)
            
            # Row count check
            cur.execute("SELECT COUNT(*) FROM transactions;")
            print(f"Transactions table uploaded successfully, {cur.fetchone()[0]} rows.")

except Exception as e:
    print("Upload failed:", e)
finally:
    conn.close()
    print("Database connection closed.")


Transactions table uploaded successfully, 1296675 rows.
Database connection closed.
